# B Point Detection Example (Empkins dataset)

## Setup and imports

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from fau_colors import cmaps

from pepbench.datasets import EmpkinsDataset
from pepbench.algorithms.icg import CPointExtractionScipyFindPeaks, BPointExtractionPale2021
from pepbench.plotting.algorithms import plot_b_point_extraction_pale2021

%matplotlib widget
%load_ext autoreload
%autoreload 2

## plotting style

In [ ]:
plt.close('all')
palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context='notebook', style='ticks', font='sans-serif', palette=palette)
plt.rcParams['figure.figsize'] = (10, 5)

## Point the dataset class to the local Empkins dataset folder inside example_data/
### This cell auto-detects the repository root so the notebook works even if Jupyter's CWD isn't the repo root.

In [ ]:
from pathlib import Path
import sys

def find_repo_root(start: Path = Path.cwd(), max_up: int = 6) -> Path | None:
    p = start
    for _ in range(max_up + 1):
        if (p / 'pyproject.toml').exists() or (p / 'src').exists() or (p / '.git').exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    return None

print('Notebook working directory:', Path.cwd())
repo_root = find_repo_root()
if repo_root is None:
    # fallback to cwd
    print('Could not find repo root by looking for pyproject.toml, src/, or .git; falling back to current working directory.')
    repo_root = Path.cwd()
else:
    print('Detected repository root:', repo_root)

# prefer example_data under the repo root
candidate = repo_root / 'example_data' / 'Empkins_Dataset'
if not candidate.exists():
    # try to locate Empkins_Dataset anywhere under repo_root (small repos this is fine)
    found = list(repo_root.rglob('Empkins_Dataset'))
    if found:
        candidate = found[0]
        print('Found Empkins_Dataset at:', candidate)

empkins_base_path = candidate
if not empkins_base_path.exists():
    raise FileNotFoundError(
        f"Empkins dataset folder not found. Tried: {candidate}\n"
        f"Please ensure the folder 'Empkins_Dataset' is located inside 'example_data/' under the repository root,\n"
        f"or set `empkins_base_path` manually to the correct absolute path. Current working dir: {Path.cwd()}"
    )

print('Using Empkins dataset path:', empkins_base_path)

## Instantiate EmpkinsDataset -- use_cache=False for reproducible load in this example

In [ ]:
dataset = EmpkinsDataset(empkins_base_path, use_cache=False, only_labeled=True)
dataset

## Show available participant/condition/phase rows and pick the first one programmatically

In [ ]:
print('Dataset index (first 10 rows):')
display(dataset.index.head(10))

## Select the first index entry (participant, condition, phase)

In [ ]:
row = dataset.index.iloc[0]
participant, condition, phase = row['participant'], row['condition'], row['phase']
print('Using:', participant, condition, phase)

## Create a subset for that single participant/condition/phase

In [ ]:
datapoint = dataset.get_subset(participant=participant, condition=condition, phase=phase)

## Run C-point extraction (used by some B-point algorithms)

In [ ]:
c_algo = CPointExtractionScipyFindPeaks()
# use the computed heartbeat segmentation
c_algo.extract(icg=datapoint.icg, heartbeats=datapoint.heartbeats, sampling_rate_hz=datapoint.sampling_rate_icg)

## Run B-point extraction (Pale et al. 2021)

In [ ]:
b_algo = BPointExtractionPale2021()
# use the same heartbeat segmentation (datapoint.heartbeats) when extracting
b_algo.extract(icg=datapoint.icg, heartbeats=datapoint.heartbeats, c_points=c_algo.points_, sampling_rate_hz=datapoint.sampling_rate_icg)

## Inspect detected points (first rows)

In [ ]:
display(b_algo.points_.head())

## Plot a few heartbeats to visualise detected B-points
### determine the first five heartbeat_ids available in the reference_heartbeats index

In [ ]:
hb_ids = list(datapoint.reference_heartbeats.index.get_level_values('heartbeat_id').unique())[:3]
print('Plotting heartbeat ids:', hb_ids)
fig, ax = plot_b_point_extraction_pale2021(datapoint, heartbeat_subset=hb_ids, normalize_time=True)
plt.show()
# TODO: why does second b point of icg look this way?

## Notes:
- You can swap `BPointExtractionPale2021` for other algorithms available in `pepbench.algorithms.icg`.
- To use non-reference heartbeats, use `datapoint.heartbeats` instead of `datapoint.reference_heartbeats`.
- If the dataset is large, set `use_cache=True` when instantiating `EmpkinsDataset` for faster repeated loads.
